## prompt test notebook

In [1]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFacePipeline
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_experimental.text_splitter import SemanticChunker
from pdf_preprocessing import LegalText

# 캐시 디렉토리 설정
cache_dir = './weights'
os.makedirs(cache_dir, exist_ok=True)
os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HF_HOME'] = cache_dir
os.environ['HF_DATASETS_CACHE'] = os.path.join(cache_dir, 'datasets')
os.environ['HUGGINGFACE_HUB_CACHE'] = cache_dir
os.environ['TORCH_HOME'] = os.path.join(cache_dir, 'torch')


/Users/minsuson/.pyenv/versions/3.10.9/envs/service-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.backends.mps.is_available():
    device = 'mps'
elif torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

def load_model(model_name):
    if model_name == 'llama':
        tokenizer = AutoTokenizer.from_pretrained(
            "davidkim205/Ko-Llama-3-8B-Instruct",
            cache_dir=cache_dir
            )
        model = AutoModelForCausalLM.from_pretrained(
            "davidkim205/Ko-Llama-3-8B-Instruct",
            device_map="auto",
            torch_dtype=torch.float16,
            cache_dir=cache_dir
            )
    elif model_name == 'qwen': 
        tokenizer = AutoTokenizer.from_pretrained(
            "davidkim205/Ko-Qwen-3-8B-Instruct",
            cache_dir=cache_dir
            )
        model = AutoModelForCausalLM.from_pretrained(
            "davidkim205/Ko-Qwen-3-8B-Instruct",
            device_map="auto",
            torch_dtype=torch.float16,
            cache_dir=cache_dir
            )
        
    return tokenizer, model

def load_embedding(model_name, device):
    if model_name == 'bge':
        model_name = "upskyy/bge-m3-korean"
        embeddings = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs={'device': device},
            encode_kwargs={'normalize_embeddings': True},
            cache_folder=cache_dir
        )
        return embeddings
    else:
        assert False, f"Unknown model name: {model_name}"


def load_doc(runpod):
    if runpod:
        pdf_path = "/workspace/LangEyE/crawling/장애인복지법.pdf"
        docs = LegalText(pdf_path).documents
    else:
        pdf_path = "/Volumes/MINDB/24년/SW아카데미/LangEyE/crawling/장애인복지법.pdf"
        docs = LegalText(pdf_path).documents
        
    return docs

In [3]:
embeddings = load_embedding('bge', device)
tokenizer, model = load_model('llama')
docs = load_doc(False)

text_splitter = SemanticChunker(embeddings)
splits = text_splitter.split_documents(docs)

/var/folders/86/f2_s35112l3f91tzt74z51m00000gn/T/ipykernel_69281/1590879090.py:37: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


KeyboardInterrupt: 

In [ ]:
vectorstore = FAISS.from_documents(
    documents=splits,
    embedding=embeddings
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.15
    )
llm = HuggingFacePipeline(pipeline=pipe)


NameError: name 'splits' is not defined

In [ ]:
template = """장애인복지법 관련 질의응답을 진행하겠습니다.

참고할 법령 내용:
{context}

질문: {question}

답변 규칙:
1. 위 법령 내용만을 기반으로 답변합니다.
2. 답변의 근거가 되는 조항(예: 제00조 제0항)을 반드시 먼저 명시합니다.
3. 조항의 내용을 직접 인용하면서 설명합니다.
4. 법령에 명시되지 않은 내용은 추정하거나 해석하지 않고, "해당 내용은 제시된 법령에 명시되어 있지 않습니다"라고 답변합니다.

답변:"""

prompt = PromptTemplate.from_template(template)


In [ ]:
def format_docs(docs):
    formatted_texts = []
    for doc in docs:
        metadata = doc.metadata
        article_number = metadata.get('article_number', '')
        paragraph = metadata.get('paragraph', '')

        formatted_text = f"{article_number} {paragraph}\n{doc.page_content}"
        formatted_texts.append(formatted_text)
    return "\n\n".join(formatted_texts)

# RAG 체인 생성 (사용자 정보 포함)
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

test_question = "장애인복지법 제3조 제1항에는 무엇이 명시되어 있나요?"
answer = rag_chain.invoke({"question": test_question})
print(test_question)
print(answer)



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# 출력 포맷팅을 위한 함수들
import re
from typing import Dict, List

def format_legal_response(response: str) -> Dict:
    """
    법률 답변을 구조화된 형태로 파싱하는 함수
    
    Returns:
        {
            'references': [참조 조항들],
            'summary': 요약 답변,
            'details': 상세 설명,
            'limitations': 한계점이나 주의사항
        }
    """
    try:
        # 참조 조항 추출 (예: 제00조 제0항, 제0호 등)
        references = re.findall(r'제\d+조(?:\s*제\d+항)?(?:\s*제\d+호)?', response)
        
        # 답변 본문에서 참조 조항 제외한 실제 설명 부분 추출
        details = response.split('\n')
        main_content = [line for line in details if not line.startswith('제')]
        
        # 한계점이나 주의사항 확인 (법령에 명시되지 않은 내용 등)
        limitations = []
        if "명시되어 있지 않습니다" in response:
            limitations.append("일부 내용은 제시된 법령에서 확인할 수 없습니다.")
            
        return {
            'references': list(set(references)),  # 중복 제거
            'summary': main_content[0] if main_content else "",
            'details': '\n'.join(main_content[1:]) if len(main_content) > 1 else "",
            'limitations': limitations
        }
    except Exception as e:
        return {
            'error': f"응답 파싱 중 오류 발생: {str(e)}",
            'raw_response': response
        }

def print_formatted_response(parsed_response: Dict):
    """
    파싱된 응답을 보기 좋게 출력하는 함수
    """
    print("=" * 80)
    print("📚 참조 조항:")
    if parsed_response.get('references'):
        for ref in parsed_response['references']:
            print(f"  • {ref}")
    else:
        print("  • 명시적 참조 조항 없음")
    
    print("\n📌 답변 요약:")
    print(f"  {parsed_response.get('summary', '요약 없음')}")
    
    if parsed_response.get('details'):
        print("\n📝 상세 설명:")
        print(parsed_response['details'])
    
    if parsed_response.get('limitations'):
        print("\n⚠️ 주의사항:")
        for limitation in parsed_response['limitations']:
            print(f"  • {limitation}")
    print("=" * 80)

# RAG 체인 수정
def process_rag_response(question: str, response: str):
    """
    RAG 체인의 응답을 처리하고 포맷팅하는 함수
    """
    parsed = format_legal_response(response)
    print(f"❓ 질문: {question}\n")
    print_formatted_response(parsed)
    return parsed

# 사용 예시:
test_question = "장애인복지법 제3조 제1항에는 무엇이 명시되어 있나요?"
answer = rag_chain.invoke({"question": test_question})
process_rag_response(test_question, answer)